In [ ]:
#SORT lists by topology and update overleaf
#keep fwd/bwd l-r labels on topology.

In [2]:
import sys
import os

sys.path.append(os.getcwd()[:-9])
from src.ops.operator import Operator
from src.ops.elemental import ElementalOperator
from src.ops.quarks import Quark
from src.ops.indexed import IndexedObject
from src.wick.contract import contract

In [3]:
q0=Quark(True,'u','s_0','c_0','t_3','x_3')
g0=IndexedObject('\\gamma_5',['s_0','s_1'])
q1=Quark(False,'d','s_1','c_0','t_3','x_3')

annihilate=Operator([ElementalOperator(+1,[g0],[q0,q1])])

elementals=[]
for pi2 in [['u',+2/3],['d',-1/3]]:
    for pi1 in [['u',+2/3],['d',-1/3]]:
        q2=Quark(True,pi2[0],'s_2','c_1','t_2','x_2')
        g1=IndexedObject('\\gamma_{\mu}',['s_2','s_3'])
        q3=Quark(False,pi2[0],'s_3','c_1','t_2','x_2')

        q4=Quark(True,pi1[0],'s_4','c_2','t_1','x_1')
        g2=IndexedObject('\\gamma_{\\nu}',['s_4','s_5'])
        q5=Quark(False,pi1[0],'s_5','c_2','t_1','x_1')

        q6=Quark(True,'d','s_6','c_3','t_0','x_0')
        g3=IndexedObject('\\gamma_5',['s_6','s_7'])
        q7=Quark(False,'u','s_7','c_3','t_0','x_0')
        elementals.append(ElementalOperator(pi2[1]*pi1[1],[g1,g2,g3],[q2,q3,q4,q5,q6,q7]))
        
create=Operator(elementals)

In [4]:
print(create)

0.4444444444444444*\gamma_{\mu}_{s_2 s_3}*\gamma_{\nu}_{s_4 s_5}*\gamma_5_{s_6 s_7}*\bar{u}_{s_2 c_1}(x_2, t_2)*u_{s_3 c_1}(x_2, t_2)*\bar{u}_{s_4 c_2}(x_1, t_1)*u_{s_5 c_2}(x_1, t_1)*\bar{d}_{s_6 c_3}(x_0, t_0)*u_{s_7 c_3}(x_0, t_0)+-0.2222222222222222*\gamma_{\mu}_{s_2 s_3}*\gamma_{\nu}_{s_4 s_5}*\gamma_5_{s_6 s_7}*\bar{u}_{s_2 c_1}(x_2, t_2)*u_{s_3 c_1}(x_2, t_2)*\bar{d}_{s_4 c_2}(x_1, t_1)*d_{s_5 c_2}(x_1, t_1)*\bar{d}_{s_6 c_3}(x_0, t_0)*u_{s_7 c_3}(x_0, t_0)+-0.2222222222222222*\gamma_{\mu}_{s_2 s_3}*\gamma_{\nu}_{s_4 s_5}*\gamma_5_{s_6 s_7}*\bar{d}_{s_2 c_1}(x_2, t_2)*d_{s_3 c_1}(x_2, t_2)*\bar{u}_{s_4 c_2}(x_1, t_1)*u_{s_5 c_2}(x_1, t_1)*\bar{d}_{s_6 c_3}(x_0, t_0)*u_{s_7 c_3}(x_0, t_0)+0.1111111111111111*\gamma_{\mu}_{s_2 s_3}*\gamma_{\nu}_{s_4 s_5}*\gamma_5_{s_6 s_7}*\bar{d}_{s_2 c_1}(x_2, t_2)*d_{s_3 c_1}(x_2, t_2)*\bar{d}_{s_4 c_2}(x_1, t_1)*d_{s_5 c_2}(x_1, t_1)*\bar{d}_{s_6 c_3}(x_0, t_0)*u_{s_7 c_3}(x_0, t_0)


In [5]:
print(annihilate)

1*\gamma_5_{s_0 s_1}*\bar{u}_{s_0 c_0}(x_3, t_3)*d_{s_1 c_0}(x_3, t_3)


In [6]:
res=contract(annihilate,create)

In [7]:
len(res)

20

In [8]:
import copy

def latexStr(gamma):
    tmp=gamma.name.split('_')
    if(tmp[0]=='\\gamma'):
        return tmp[0] + '_{' + tmp[1] + '_{' + gamma.indices[0] + gamma.indices[1] + '}}'
    else:
        return str(tmp)
  
def platexStr(p):
    return p.name + '(' + p.ti.replace('t','x') + p.ti + '\mid ' + p.tf.replace('t','x') + p.tf + ')' + '_{\\substack{' + p.left_indices.s + '\\\\' + p.left_indices.c + '}' +  '\\substack{' + p.right_indices.s + '\\\\' + p.right_indices.c + '}}'
    
total=copy.deepcopy(res)

print(len(total))

for d in total:
    for p in d.props:
        #p.name = 'S'
        p.name = 'S^'+p.name.split('^')[1]
        
for i in range(len(total)-1,-1,-1):
    for j in range(len(total)-1,-1,-1):
        if(i!=j):
            if((total[i].commuting==total[j].commuting) and (total[i].props==total[j].props)):
            #if(total[i]==total[j]):
                total[i].coef+=total[j].coef
                total.pop(j)
                break
for i in range(len(total)-1,-1,-1):
    if(total[i].coef==0):
        total.pop(i)
    else:
        total[i].coef*=9
        
print(len(total))

20
20


In [9]:
def print_diagrams(total_diags):
    print('\\beqs')
    for i,d in enumerate(total_diags):
        #if(i==1 or i==6):
        tmp='d_{'+str(i)+'}='+str(d.coef) + ' '
        for p in d.props:
            tmp+=platexStr(p)
        for c in d.commuting:
            tmp+=latexStr(c)
        print(tmp+'\\\\')
    print('\\eeqs')
    

In [10]:
print_diagrams(total)

\beqs
d_{0}=4.0 S^u(x_0t_0\mid x_3t_3)_{\substack{s_7\\c_3}\substack{s_0\\c_0}}S^u(x_1t_1\mid x_2t_2)_{\substack{s_5\\c_2}\substack{s_2\\c_1}}S^u(x_2t_2\mid x_1t_1)_{\substack{s_3\\c_1}\substack{s_4\\c_2}}S^d(x_3t_3\mid x_0t_0)_{\substack{s_1\\c_0}\substack{s_6\\c_3}}\gamma_{5_{s_0s_1}}\gamma_{5_{s_6s_7}}\gamma_{{\mu}_{s_2s_3}}\gamma_{{\nu}_{s_4s_5}}\\
d_{1}=-4.0 S^u(x_1t_1\mid x_3t_3)_{\substack{s_5\\c_2}\substack{s_0\\c_0}}S^u(x_0t_0\mid x_2t_2)_{\substack{s_7\\c_3}\substack{s_2\\c_1}}S^u(x_2t_2\mid x_1t_1)_{\substack{s_3\\c_1}\substack{s_4\\c_2}}S^d(x_3t_3\mid x_0t_0)_{\substack{s_1\\c_0}\substack{s_6\\c_3}}\gamma_{5_{s_0s_1}}\gamma_{5_{s_6s_7}}\gamma_{{\mu}_{s_2s_3}}\gamma_{{\nu}_{s_4s_5}}\\
d_{2}=4.0 S^u(x_2t_2\mid x_3t_3)_{\substack{s_3\\c_1}\substack{s_0\\c_0}}S^u(x_0t_0\mid x_2t_2)_{\substack{s_7\\c_3}\substack{s_2\\c_1}}S^u(x_1t_1\mid x_1t_1)_{\substack{s_5\\c_2}\substack{s_4\\c_2}}S^d(x_3t_3\mid x_0t_0)_{\substack{s_1\\c_0}\substack{s_6\\c_3}}\gamma_{5_{s_0s_1}}\gamma_{5_{s_6

In [11]:
def diagram_time_order(total_diags,log=False):
    diagrams = {}

    for i,d in enumerate(total):
        top = []
        d_str='d_' + str(i) + ':  '
        for p in d.props:
            d_str+=p.tf + '->' + p.ti + '   '
            top.append([p.tf.split('_')[1],p.ti.split('_')[1]])
        diagrams[i]=top
        if(log):
            print(d_str)
    return diagrams
    

In [12]:
refTopologies={}
refTopologies['A']=[['1','0'],['0','2'],['2','3'],['3','1']]
refTopologies['A-bwd']=[['0','1'],['1','3'],['3','2'],['2','0']]
refTopologies['B']=[['1','0'],['0','3'],['3','2'],['2','1']]
refTopologies['B-bwd']=[['0','1'],['1','2'],['2','3'],['3','0']]
refTopologies['C']=[['2','0'],['0','3'],['3','1'],['1','2']]
refTopologies['C-bwd']=[['0','2'],['2','1'],['1','3'],['3','0']]
refTopologies['D']=[['0','3'],['3','0'],['2','1'],['1','2']]
refTopologies['El']=[['2','2'],['3','1'],['1','0'],['0','3']]
refTopologies['El-bwd']=[['2','2'],['1','3'],['3','0'],['0','1']]
refTopologies['Er']=[['1','1'],['3','2'],['2','0'],['0','3']]
refTopologies['Er-bwd']=[['1','1'],['2','3'],['3','0'],['0','2']]
refTopologies['F']=[['0','3'],['3','0'],['1','1'],['2','2']]

In [13]:
from src.wick.utilities import permutations

def swap(list, i, j):
    list[i], list[j] = list[j], list[i]
    return list

def cyclic_permutations(list):
    return [[list[i-j] for i in range(len(list))] for j in range(len(list))]

#I kind of forget what this does...
def connect_times(topList):
    for idx,d in topList.items():
        for p in range(len(d)):
            if d[p][1] != d[(p+1)%len(d)][0]:
                for pp in range(p,len(d)):
                    if d[p][1] == d[pp][0]:
                        swap(d,(p+1)%len(d),pp)

def name_topologies(topList):
    for idx,d in topList.items():
        for label,top in refTopologies.items():
            if d in permutations(top):
                topList[idx]=label

In [14]:
topList=diagram_time_order(total)

connect_times(topList)

name_topologies(topList)

In [15]:
topList

{0: 'D',
 1: 'C',
 2: 'Er',
 3: 'F',
 4: 'El',
 5: 'B',
 6: 'El-bwd',
 7: 'A-bwd',
 8: 'Er',
 9: 'F',
 10: 'A',
 11: 'Er-bwd',
 12: 'F',
 13: 'El',
 14: 'El-bwd',
 15: 'B-bwd',
 16: 'Er-bwd',
 17: 'C-bwd',
 18: 'D',
 19: 'F'}

In [23]:
x=copy.deepcopy(topList)
topSorted={k: v for k, v in sorted(x.items(), key=lambda item: item[1])}
print(topSorted)

{10: 'A', 7: 'A-bwd', 5: 'B', 15: 'B-bwd', 1: 'C', 17: 'C-bwd', 0: 'D', 18: 'D', 4: 'El', 13: 'El', 6: 'El-bwd', 14: 'El-bwd', 2: 'Er', 8: 'Er', 11: 'Er-bwd', 16: 'Er-bwd', 3: 'F', 9: 'F', 12: 'F', 19: 'F'}


In [24]:
def print_diagrams_labelled(diags,sortd):
    print('\\beqs')
    newI=0
    for i,label in sortd.items():
        #if(i==1 or i==6):
        tmp='d_{'+str(i)+'}^{\\text{('+label+')}}='+str(diags[i].coef) + ' '
        for p in diags[i].props:
            tmp+=platexStr(p)
        for c in diags[i].commuting:
            tmp+=latexStr(c)
        print(tmp+'\\\\')
        newI+=1
    print('\\eeqs')

In [25]:
print_diagrams_labelled(total,topSorted)

\beqs
d_{10}^{\text{(A)}}=2.0 S^u(x_1t_1\mid x_3t_3)_{\substack{s_5\\c_2}\substack{s_0\\c_0}}S^d(x_3t_3\mid x_2t_2)_{\substack{s_1\\c_0}\substack{s_2\\c_1}}S^u(x_0t_0\mid x_1t_1)_{\substack{s_7\\c_3}\substack{s_4\\c_2}}S^d(x_2t_2\mid x_0t_0)_{\substack{s_3\\c_1}\substack{s_6\\c_3}}\gamma_{5_{s_0s_1}}\gamma_{5_{s_6s_7}}\gamma_{{\mu}_{s_2s_3}}\gamma_{{\nu}_{s_4s_5}}\\
d_{7}^{\text{(A-bwd)}}=2.0 S^u(x_2t_2\mid x_3t_3)_{\substack{s_3\\c_1}\substack{s_0\\c_0}}S^u(x_0t_0\mid x_2t_2)_{\substack{s_7\\c_3}\substack{s_2\\c_1}}S^d(x_3t_3\mid x_1t_1)_{\substack{s_1\\c_0}\substack{s_4\\c_2}}S^d(x_1t_1\mid x_0t_0)_{\substack{s_5\\c_2}\substack{s_6\\c_3}}\gamma_{5_{s_0s_1}}\gamma_{5_{s_6s_7}}\gamma_{{\mu}_{s_2s_3}}\gamma_{{\nu}_{s_4s_5}}\\
d_{5}^{\text{(B)}}=-4.0 S^u(x_2t_2\mid x_3t_3)_{\substack{s_3\\c_1}\substack{s_0\\c_0}}S^u(x_1t_1\mid x_2t_2)_{\substack{s_5\\c_2}\substack{s_2\\c_1}}S^u(x_0t_0\mid x_1t_1)_{\substack{s_7\\c_3}\substack{s_4\\c_2}}S^d(x_3t_3\mid x_0t_0)_{\substack{s_1\\c_0}\substack